# 라즈베리파이 실시간 전압/전류 측정

## ⚙ 구성
|항목|모델명|용도|
|-------|-------|-------|
전압 측정|ZMPT101B|220V 전압 감지(출력: 아날로그)
전류 측정|ACS712 (5A/20A/30A)|부하 전류 감지 (출력: 아날로그)
ADC|ADS1115|아날로그 센서 → 디지털 신호로 변환
저장 형식|CSV|전처리 용이, 속도 빠름
측정 주기|100Hz(10ms 간격, 일반 논문에서 자주 사용)

## 🧠 흐름
- ZMPT101B & ACS712 (Analog)
- ADS1115 (I2C → Raspberry Pi)
- 실시간 측정 (0.01초 간격)
- CSV 파일로 저장

## ⚠️ 참고사항
- ACS712 전류 변환 공식: `current (A) = (measured_voltage - Vcc/2) / sensitivity` (예: 20A 모델은 100mV/A, Vcc=5V일 때 2.5V 중심 기준)
- ZMPT101B도 보정이 필요하지만, 일단 상대적 변화 감지용으로도 사용 가능

In [ ]:
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import pandas as pd
import numpy as np


In [7]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c)
ads.gain = 2/3  # ±6.144V (ADC 입력 범위)

# 센서 채널 지정
voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

# 센서 기준 오프셋
ZMPT_offset = 2.554
ZMPT_scale = 1000.0     # 실험을 통해 보정 예정

ACS712_offset = 2.508
ACS712_sensitivity = 0.1  # 또는 0.05

In [6]:
data = []
count = 0

# 이전 정상값을 저장할 변수
prev_v_raw = None
prev_c_raw = None

try:
    while True:
        now = time.time()
        error_flag = 0  # 기본은 정상

        try:
            v_raw = voltage_ch.voltage
            c_raw = current_ch.voltage
            prev_v_raw = v_raw  # 정상 측정 시 업데이트
            prev_c_raw = c_raw
        except Exception as e:
            print(f"[I2C 오류 발생] {e}")
            v_raw = prev_v_raw if prev_v_raw is not None else np.nan
            c_raw = prev_c_raw if prev_c_raw is not None else np.nan
            error_flag = 1

        if not np.isnan(v_raw) and not np.isnan(c_raw):
            real_voltage = (v_raw - ZMPT_offset) * ZMPT_scale
            real_current = (c_raw - ACS712_offset) / ACS712_sensitivity
        else:
            real_voltage = np.nan
            real_current = np.nan

        data.append([now, v_raw, c_raw, real_voltage, real_current, error_flag])
        count += 1

        # 매 100개마다 CSV로 저장 및 보정 확인 로그 출력
        if len(data) >= 100:
            df = pd.DataFrame(data, columns=[
                'timestamp', 'v_raw', 'c_raw', 'voltage(V)', 'current(A)', 'error_flag'
            ])

            df.to_csv("raw_dataset_arc.csv", mode='a', index=False, header=False)

            print(f"[{count}] V_raw: {df['v_raw'].mean():.3f}, C_raw: {df['c_raw'].mean():.3f}")
            print(f"      Voltage(V): {df['voltage(V)'].mean():.2f}, Current(A): {df['current(A)'].mean():.2f}")
            print(f"      Error Count: {df['error_flag'].sum()}")
            print("-" * 60)

            data = []

        time.sleep(0.001163)  # 860Hz 간격

except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")

[100] V_raw: 2.551, C_raw: 2.507
      Voltage(V): -2.84, Current(A): -0.01
      Error Count: 0
------------------------------------------------------------
[200] V_raw: 2.550, C_raw: 2.508
      Voltage(V): -3.79, Current(A): 0.00
      Error Count: 0
------------------------------------------------------------
[300] V_raw: 2.554, C_raw: 2.508
      Voltage(V): 0.48, Current(A): -0.00
      Error Count: 0
------------------------------------------------------------
[400] V_raw: 2.553, C_raw: 2.507
      Voltage(V): -0.79, Current(A): -0.01
      Error Count: 0
------------------------------------------------------------
[500] V_raw: 2.547, C_raw: 2.503
      Voltage(V): -7.00, Current(A): -0.05
      Error Count: 0
------------------------------------------------------------
[600] V_raw: 2.548, C_raw: 2.504
      Voltage(V): -5.99, Current(A): -0.04
      Error Count: 0
------------------------------------------------------------
[700] V_raw: 2.548, C_raw: 2.504
      Voltage(V): -6.